In [ ]:
import os
from PIL import Image

path = '/workspace/train_original/image'
crop_path = '/workspace/train_original/crop_padding_image'
list = os.listdir(path)

In [ ]:
from ultralytics import YOLO

# 1. 모델 로드:
model = YOLO('/workspace/yolo_weights/best.pt') 

# 2. 예측할 이미지 경로:
def inference_image(file_name):
    img = os.path.join(path, file_name)
    # 3. 예측 실행:
    result = model.predict(source=img, save=False, conf=0.6, max_det=1)[0]
    
    # YOLO가 들고있는 원본 이미지 (numpy, 보통 BGR)
    orig_np = result.orig_img   # shape: (H, W, 3)

    # BGR -> RGB 변환 후 PIL로
    orig_rgb = orig_np[:, :, ::-1]  # BGR -> RGB
    orig_pil = Image.fromarray(orig_rgb)

    img_w, img_h = orig_pil.size

    # bbox 좌표 가져오기 (x1, y1, x2, y2)
    boxes = result.boxes.xyxy.cpu().numpy().astype(int)

    if len(boxes) == 0:
        return orig_pil, None
    
    x1, y1, x2, y2 = boxes[0]

    # 패딩 추가 및 경계 처리 (Clamping) 로직
    padding = 60

    # 좌표 보정
    # max(0, ...) : 0보다 작아지는 것 방지 (왼쪽, 위쪽 경계)
    # min(길이, ...) : 이미지 크기보다 커지는 것 방지 (오른쪽, 아래쪽 경계)
    p_x1 = max(0, x1 - padding)
    p_y1 = max(0, y1 - padding)
    p_x2 = min(img_w, x2 + padding)
    p_y2 = min(img_h, y2 + padding)
    
    # PIL crop (왼쪽, 위, 오른쪽, 아래)
    crop_pil = orig_pil.crop((p_x1, p_y1, p_x2, p_y2))
    crop_pil.save(os.path.join(crop_path, file_name))
    
for i in list:
    if os.path.exists(os.path.join(crop_path, i)):
        continue
    else:
        inference_image(i)
